In [1]:
!pip install langchain-community langchain-chroma sentence-transformers

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


## Embed Data into Vector Store
The below code assumes that the huggingface dataset has the complete chunks in one column named "text".

In [5]:
from retriever import Retriever

# store = Retriever("slyq/wdc-products-chunked", "combined")
# store = Retriever("slyq/code-chunked")
store = Retriever("slyq/papers-chunked")

/home/slyq/miniconda3/envs/splitter/lib/python3.9/site-packages/datasets/load.py:2547: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(
Generating train split: 100%|██████████| 397/397 [00:00<00:00, 11579.87 examples/s]


We can optionally test if RAG is working properly.

In [3]:
# docs = store.query("How much RAM does an Apple Macbook Pro have?")
# print(docs[0].page_content)

"\"MacBook Pro with Touch Bar 15-inch 256GB (2017) 256gb Space Gray\"  \"Apple MacBook Pro - 13.3\" Core i5 4 GB RAM 500 HDD English\" \" Apple English MD101LL/A Notebook Computers CDW.com;\n\"MacBook Pro is machined from a single piece of aluminum, an engineering breakthrough that replaced many parts with just one. It's called the unibody. And the first time you pick up a MacBook Pro, you'll notice the difference it makes. The entire enclosure is thinner and lighter than other notebooks. It looks polished and refined. And it feels strong and durable - perfect for life inside (and outside) your briefcase or backpack.\" ;"


## Augment Eval Dataset with Documents from RAG

In [6]:
from datasets import load_dataset

raw_dataset = load_dataset("scottsus/papers-test", split="test")

def augment(data):
    data["contexts"] = [[doc.page_content for doc in store.query(d)] for d in data["question"]]
    return data

eval_dataset = raw_dataset.map(augment, batched=True)

Map: 100%|██████████| 1976/1976 [07:29<00:00,  4.39 examples/s]


In [7]:
print(eval_dataset)
print(eval_dataset[0])

Dataset({
    features: ['choices', 'answer', 'question', 'contexts'],
    num_rows: 1976
})
{'choices': ['A. Focused on numeric data processing only', 'B. Strong text encoding/decoding ability and reasoning', 'C. Graph visualization techniques', 'D. Purely academic data structuring'], 'answer': 'B', 'question': 'What is a significant contribution of large language models such as GPT4 and LLaMA in the field of natural language processing?', 'contexts': ['"Large Language Models: A Survey\\n                              Shervin Minaee, Tomas Mikolov, Narjes Nikzad, Meysam Chenaghlu\\n                                            Richard Socher, Xavier Amatriain, Jianfeng Gao\\n   Abstract\\u2014Large  Language  Models  (LLMs)  have  drawn  a                     that have different starting points and velocity: statistical lan-\\nlot  of  attention  due  to  their  strong  performance  on  a  wide            guage models, neural language models, pre-trained language\\nrange of natural langu

In [9]:
eval_dataset.push_to_hub("slyq/papers-mcq", "ragged", split="test")

Uploading the dataset shards: 100%|██████████| 1/1 [00:00<00:00,  2.98it/s]


CommitInfo(commit_url='https://huggingface.co/datasets/slyq/papers-mcq/commit/064fe9fdb568d27b613d41910c64d9a75f5e613e', commit_message='Upload dataset', commit_description='', oid='064fe9fdb568d27b613d41910c64d9a75f5e613e', pr_url=None, pr_revision=None, pr_num=None)